<a href="https://colab.research.google.com/github/27vamsi/PI-Neural-Networks/blob/main/PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import torch
import os
import gc
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.linalg import block_diag

class YOLOv8FireDetector:
    def __init__(self, model_path='bestt.pt'):
        print(f"Initializing YOLOv8 detector with model: {model_path}")

        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found at {model_path}")

        try:
            import ultralytics
        except ImportError:
            print("Installing ultralytics...")
            !pip install -q ultralytics
            import ultralytics

        from ultralytics import YOLO
        self.model = YOLO(model_path)

        self.conf_threshold = 0.05

        self.model_path = model_path
        self.model_name = os.path.basename(model_path)

        self.metrics = {
            'inference_times': [],
            'detection_counts': [],
            'confidence_scores': [],
            'fire_areas': [],
            'fire_intensities': [],
            'iou_scores': []
        }

        self.detection_history = []
        self.max_history_length = 10

        self.prev_detections = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name'])

        self.position_stability_threshold = 5
        self.color_variance_threshold = 20
        self.min_shape_change_threshold = 0.05

        self.ceiling_filter_enabled = True
        self.ceiling_height_threshold = 0.25
        self.center_width_range = (0.25, 0.75)

    def calculate_iou(self, box1, box2):
        x1_inter = max(box1[0], box2[0])
        y1_inter = max(box1[1], box2[1])
        x2_inter = min(box1[2], box2[2])
        y2_inter = min(box1[3], box2[3])

        if x2_inter < x1_inter or y2_inter < y1_inter:
            return 0.0

        inter_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)

        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

        union_area = box1_area + box2_area - inter_area

        return inter_area / union_area if union_area > 0 else 0.0

    def detect(self, frame):
        start_time = time.time()

        results = self.model(frame, conf=self.conf_threshold, verbose=False)

        inference_time = time.time() - start_time
        self.metrics['inference_times'].append(inference_time)

        mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)

        detections = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name'])

        detection_count = 0
        confidence_scores = []
        fire_area = 0
        avg_intensity = 0

        if len(results) > 0:
            result = results[0]

            if len(result.boxes) > 0:
                boxes = result.boxes

                if boxes.xyxy.numel() > 0:
                    xyxy = boxes.xyxy.cpu().numpy()
                    conf = boxes.conf.cpu().numpy()
                    cls = boxes.cls.cpu().numpy()

                    detections_list = []
                    for i in range(len(xyxy)):
                        x1, y1, x2, y2 = xyxy[i]
                        detections_list.append({
                            'xmin': x1,
                            'ymin': y1,
                            'xmax': x2,
                            'ymax': y2,
                            'confidence': conf[i],
                            'class': cls[i],
                            'name': result.names[int(cls[i])]
                        })

                    initial_detections = pd.DataFrame(detections_list)

                    filtered_detections = self.filter_lamp_detections(frame, initial_detections)
                    detections = filtered_detections

                    detection_count = len(detections)
                    confidence_scores = detections['confidence'].tolist()

                    total_intensity = 0
                    pixel_count = 0

                    for _, detection in detections.iterrows():
                        x1, y1, x2, y2 = int(detection['xmin']), int(detection['ymin']), int(detection['xmax']), int(detection['ymax'])
                        area = (x2 - x1) * (y2 - y1)
                        fire_area += area

                        if x1 < frame.shape[1] and y1 < frame.shape[0] and x2 > 0 and y2 > 0:
                            x1 = max(0, x1)
                            y1 = max(0, y1)
                            x2 = min(frame.shape[1], x2)
                            y2 = min(frame.shape[0], y2)

                            if len(frame.shape) == 3:
                                fire_region = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
                            else:
                                fire_region = frame[y1:y2, x1:x2]

                            if fire_region.size > 0:
                                total_intensity += np.sum(fire_region)
                                pixel_count += fire_region.size

                            cv2.rectangle(
                                mask,
                                (x1, y1),
                                (x2, y2),
                                255,
                                -1
                            )

                    if pixel_count > 0:
                        avg_intensity = total_intensity / pixel_count

        self.metrics['detection_counts'].append(detection_count)
        self.metrics['confidence_scores'].extend(confidence_scores)
        self.metrics['fire_areas'].append(fire_area)
        self.metrics['fire_intensities'].append(avg_intensity)

        if not self.prev_detections.empty and not detections.empty:
            iou_scores = []
            for _, curr_det in detections.iterrows():
                curr_box = [curr_det['xmin'], curr_det['ymin'], curr_det['xmax'], curr_det['ymax']]
                best_iou = 0
                for _, prev_det in self.prev_detections.iterrows():
                    prev_box = [prev_det['xmin'], prev_det['ymin'], prev_det['xmax'], prev_det['ymax']]
                    iou = self.calculate_iou(curr_box, prev_box)
                    best_iou = max(best_iou, iou)
                iou_scores.append(best_iou)

            if iou_scores:
                self.metrics['iou_scores'].append(np.mean(iou_scores))

        self.prev_detections = detections.copy()

        return mask, detections

    def filter_lamp_detections(self, frame, detections, iou_threshold=0.4):
        if detections.empty:
            return detections

        real_fire_indices = []

        for idx, detection in detections.iterrows():
            x1, y1, x2, y2 = int(detection['xmin']), int(detection['ymin']), int(detection['xmax']), int(detection['ymax'])

            if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0 or x2 >= frame.shape[1] or y2 >= frame.shape[0]:
                continue

            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2

            rel_y = center_y / frame.shape[0]
            rel_x = center_x / frame.shape[1]

            is_ceiling_lamp = False
            if self.ceiling_filter_enabled:
                near_ceiling = rel_y < self.ceiling_height_threshold
                in_center = self.center_width_range[0] < rel_x < self.center_width_range[1]
                is_ceiling_lamp = near_ceiling and in_center

                is_small = (x2 - x1) * (y2 - y1) < (0.05 * frame.shape[0] * frame.shape[1])
                if is_ceiling_lamp and is_small:
                    continue

            if not is_ceiling_lamp:
                roi = frame[y1:y2, x1:x2]
                color_variance = self._check_color_variance(roi)

                has_fire_colors = color_variance > self.color_variance_threshold

                if has_fire_colors or not near_ceiling:
                    real_fire_indices.append(idx)

        if hasattr(self, 'prev_detections') and not self.prev_detections.empty:
            for idx, detection in detections.iterrows():
                if idx in real_fire_indices:
                    continue  # Already identified as fire

                curr_box = [detection['xmin'], detection['ymin'], detection['xmax'], detection['ymax']]

                has_temporal_match = False
                for _, prev_det in self.prev_detections.iterrows():
                    prev_box = [prev_det['xmin'], prev_det['ymin'], prev_det['xmax'], prev_det['ymax']]
                    iou = self.calculate_iou(curr_box, prev_box)

                    if iou >= iou_threshold:
                        has_temporal_match = True
                        break

                if has_temporal_match and idx not in real_fire_indices:
                    real_fire_indices.append(idx)

        self.detection_history.append({
            'frame': frame.copy() if len(self.detection_history) < 2 else None,
            'detections': detections.copy(),
            'timestamp': time.time()
        })

        if len(self.detection_history) > self.max_history_length:
            self.detection_history.pop(0)

        return detections.iloc[real_fire_indices].reset_index(drop=True)

    def _check_color_variance(self, roi):
        if roi is None or roi.size == 0:
            return 0

        try:
            if roi.shape[0] == 0 or roi.shape[1] == 0:
                return 0

            hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

            h_var = np.var(hsv_roi[:,:,0])
            s_var = np.var(hsv_roi[:,:,1])
            v_var = np.var(hsv_roi[:,:,2])

            r_var = np.var(roi[:,:,2])
            g_var = np.var(roi[:,:,1])
            b_var = np.var(roi[:,:,0])

            r_mean = np.mean(roi[:,:,2])
            g_mean = np.mean(roi[:,:,1])
            b_mean = np.mean(roi[:,:,0])

            red_dominance = r_mean / (g_mean + b_mean + 1e-6)

            color_variance = (v_var * 0.3 + r_var * 0.3 + h_var * 0.2 + s_var * 0.1 + red_dominance * 10)

            return color_variance
        except Exception as e:
            return 0

    def _check_position_stability(self, x1, y1, x2, y2):
        current_center_x = (x1 + x2) / 2
        current_center_y = (y1 + y2) / 2

        centers = []

        for hist_item in self.detection_history[:-1]:
            for _, hist_det in hist_item['detections'].iterrows():
                hist_x1, hist_y1 = int(hist_det['xmin']), int(hist_det['ymin'])
                hist_x2, hist_y2 = int(hist_det['xmax']), int(hist_det['ymax'])

                hist_center_x = (hist_x1 + hist_x2) / 2
                hist_center_y = (hist_y1 + hist_y2) / 2

                center_distance = np.sqrt((current_center_x - hist_center_x)**2 +
                                         (current_center_y - hist_center_y)**2)

                if center_distance < 50:
                    centers.append((hist_center_x, hist_center_y))
                    break

        if len(centers) < 2:
            return False

        center_x_values = [c[0] for c in centers]
        center_y_values = [c[1] for c in centers]

        x_variance = np.var(center_x_values)
        y_variance = np.var(center_y_values)

        return (x_variance < self.position_stability_threshold and
                y_variance < self.position_stability_threshold)

    def get_contour_info(self, mask):
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if not contours:
            return None, 0, (0, 0)

        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)

        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0

        return largest_contour, area, (cx, cy)

    def analyze_contour_shape(self, contour):
        if contour is None or len(contour) < 5:
            return None, 0

        try:
            ellipse = cv2.fitEllipse(contour)
            center, axes, angle = ellipse

            angle_rad = np.deg2rad(angle)
            direction_x = np.cos(angle_rad)
            direction_y = np.sin(angle_rad)

            major_axis, minor_axis = axes

            if major_axis > 0 and minor_axis > 0 and major_axis >= minor_axis:
                try:
                    eccentricity = np.sqrt(max(0, 1 - (minor_axis/major_axis)**2))
                except:
                    eccentricity = 0
            else:
                eccentricity = 0

            return (direction_x, direction_y), eccentricity
        except:
            return None, 0

    def extract_intensity_gradient(self, frame, mask):
        if len(frame.shape) == 3:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:
            gray = frame.copy()

        masked_frame = cv2.bitwise_and(gray, gray, mask=mask)

        if np.sum(mask) <= 0:
            return (0, 0), 0

        grad_x = cv2.Sobel(masked_frame, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(masked_frame, cv2.CV_64F, 0, 1, ksize=3)

        try:
            avg_grad_x = np.mean(grad_x[mask > 0])
            avg_grad_y = np.mean(grad_y[mask > 0])

            magnitude = np.sqrt(avg_grad_x**2 + avg_grad_y**2)

            if magnitude > 1e-6:
                avg_grad_x /= magnitude
                avg_grad_y /= magnitude
            else:
                avg_grad_x, avg_grad_y = 0, 0
        except:
            return (0, 0), 0

        return (avg_grad_x, avg_grad_y), magnitude

    def get_performance_metrics(self):
        metrics = {}

        if self.metrics['inference_times']:
            metrics['avg_inference_time'] = np.mean(self.metrics['inference_times'])
            metrics['min_inference_time'] = np.min(self.metrics['inference_times'])
            metrics['max_inference_time'] = np.max(self.metrics['inference_times'])
            metrics['fps'] = 1.0 / metrics['avg_inference_time']
            metrics['total_frames'] = len(self.metrics['inference_times'])
        else:
            metrics['avg_inference_time'] = 0
            metrics['fps'] = 0
            metrics['total_frames'] = 0

        metrics['total_detections'] = sum(self.metrics['detection_counts'])
        if metrics['total_frames'] > 0:
            metrics['avg_detections_per_frame'] = metrics['total_detections'] / metrics['total_frames']
        else:
            metrics['avg_detections_per_frame'] = 0

        frames_with_detections = sum(1 for count in self.metrics['detection_counts'] if count > 0)
        metrics['frames_with_detections'] = frames_with_detections
        metrics['detection_rate'] = frames_with_detections / metrics['total_frames'] if metrics['total_frames'] > 0 else 0

        if self.metrics['confidence_scores']:
            metrics['avg_confidence'] = np.mean(self.metrics['confidence_scores'])
            metrics['min_confidence'] = np.min(self.metrics['confidence_scores'])
            metrics['max_confidence'] = np.max(self.metrics['confidence_scores'])
        else:
            metrics['avg_confidence'] = 0
            metrics['min_confidence'] = 0
            metrics['max_confidence'] = 0

        if self.metrics['fire_areas']:
            metrics['avg_fire_area'] = np.mean(self.metrics['fire_areas'])
            metrics['max_fire_area'] = np.max(self.metrics['fire_areas'])

            non_zero_areas = [area for area in self.metrics['fire_areas'] if area > 0]
            if len(non_zero_areas) > 1:
                growth_rates = [non_zero_areas[i] / non_zero_areas[i-1] if non_zero_areas[i-1] > 0 else 1
                               for i in range(1, len(non_zero_areas))]
                if growth_rates:
                    metrics['avg_growth_rate'] = np.mean(growth_rates)
                    metrics['max_growth_rate'] = np.max(growth_rates)

        if self.metrics['iou_scores']:
            metrics['avg_iou'] = np.mean(self.metrics['iou_scores'])
            metrics['max_iou'] = np.max(self.metrics['iou_scores'])
            metrics['min_iou'] = np.min(self.metrics['iou_scores'])

        return metrics


class KalmanFilter:
    def __init__(self):
        self.state_dim = 6
        self.measurement_dim = 2

        self.x = np.zeros((self.state_dim, 1))

        self.F = np.eye(self.state_dim)

        self.H = np.zeros((self.measurement_dim, self.state_dim))
        self.H[0, 0] = 1
        self.H[1, 1] = 1

        self.Q = np.eye(self.state_dim) * 0.01

        self.R = np.eye(self.measurement_dim) * 10

        self.P = np.eye(self.state_dim) * 100

        self.I = np.eye(self.state_dim)

        self.initialized = False

    def update_transition_matrix(self, dt):
        self.F[0, 2] = dt
        self.F[0, 4] = 0.5*dt*dt
        self.F[1, 3] = dt
        self.F[1, 5] = 0.5*dt*dt
        self.F[2, 4] = dt
        self.F[3, 5] = dt

    def init(self, measurement):
        x, y = measurement
        self.x = np.array([[x], [y], [0], [0], [0], [0]])
        self.initialized = True

    def predict(self, dt=1.0):
        if not self.initialized:
            return (0, 0)

        self.update_transition_matrix(dt)

        self.x = np.dot(self.F, self.x)

        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q

        return (self.x[0, 0], self.x[1, 0])

    def update(self, measurement):
        if not self.initialized:
            self.init(measurement)
            return

        z = np.array([[measurement[0]], [measurement[1]]])

        S = np.dot(np.dot(self.H, self.P), self.H.T) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))

        y = z - np.dot(self.H, self.x)
        self.x = self.x + np.dot(K, y)

        self.P = np.dot((self.I - np.dot(K, self.H)), self.P)

    def get_state(self):
        if not self.initialized:
            return (0, 0), (0, 0), (0, 0)

        pos = (self.x[0, 0], self.x[1, 0])
        vel = (self.x[2, 0], self.x[3, 0])
        acc = (self.x[4, 0], self.x[5, 0])

        return pos, vel, acc

    def predict_future(self, steps_ahead):
        if not self.initialized:
            return [(0, 0)] * steps_ahead

        predictions = []
        current_state = self.x.copy()
        current_transition = self.F.copy()

        for i in range(steps_ahead):
            current_state = np.dot(current_transition, current_state)
            predictions.append((current_state[0, 0], current_state[1, 0]))

        return predictions


class FireSpreadPredictor:
    def __init__(self, yolo_model_path='bestt.pt', history_size=5, prediction_window=10):
        self.history_size = history_size
        self.prediction_window = prediction_window
        self.position_history = []
        self.area_history = []
        self.contour_history = []
        self.detections_history = []
        self.intensity_history = []
        self.shape_history = []
        self.gradient_history = []
        self.iou_history = []  # Added for IOU tracking

        self.feature_weights = {
            'position': 0.4,
            'shape': 0.2,
            'intensity': 0.2,
            'gradient': 0.2
        }

        self.dt = 1.0
        self.last_frame_time = None

        self.kalman = KalmanFilter()

        self.predictions = []
        self.prediction_results = []
        self.prediction_stats = {
            'total_predictions': 0,
            'successful_predictions': 0,
            'failed_predictions': 0,
            'success_rate': 0.0,
            'prediction_errors': [],
            'thresholds_used': []
        }

        self.quality_categories = {
            'excellent': 0,
            'good': 0,
            'moderate': 0,
            'poor': 0
        }

        self.fire_detector = YOLOv8FireDetector(model_path=yolo_model_path)
        self.previous_detections = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name'])
        print("Enhanced FireSpreadPredictor initialized with ceiling lamp filtering and IOU tracking")

    def track_detections_with_iou(self, current_detections, previous_detections, iou_threshold=0.3):
        if previous_detections.empty or current_detections.empty:
            return []

        matches = []

        for curr_idx, curr_det in current_detections.iterrows():
            curr_box = [curr_det['xmin'], curr_det['ymin'], curr_det['xmax'], curr_det['ymax']]
            best_iou = 0
            best_match = -1

            for prev_idx, prev_det in previous_detections.iterrows():
                prev_box = [prev_det['xmin'], prev_det['ymin'], prev_det['xmax'], prev_det['ymax']]
                iou = self.fire_detector.calculate_iou(curr_box, prev_box)

                if iou > best_iou and iou >= iou_threshold:
                    best_iou = iou
                    best_match = prev_idx

            if best_match != -1:
                matches.append((curr_idx, best_match, best_iou))

        return matches

    def update_kalman_with_best_match(self, iou_matches, current_detections, previous_detections):
        if not iou_matches:
            return

        sorted_matches = sorted(iou_matches, key=lambda x: x[2], reverse=True)
        best_match = sorted_matches[0]

        curr_idx = best_match[0]
        curr_det = current_detections.iloc[curr_idx]

        center_x = (curr_det['xmin'] + curr_det['xmax']) / 2
        center_y = (curr_det['ymin'] + curr_det['ymax']) / 2

        self.kalman.update((center_x, center_y))

    def add_frame(self, frame, frame_index, timestamp=None):
        if timestamp is not None:
            if self.last_frame_time is not None:
                self.dt = timestamp - self.last_frame_time
            self.last_frame_time = timestamp

        mask, detections = self.fire_detector.detect(frame)
        contour, area, centroid = self.fire_detector.get_contour_info(mask)

        shape_direction, shape_confidence = (0, 0), 0
        if contour is not None:
            shape_direction, shape_confidence = self.fire_detector.analyze_contour_shape(contour)

        gradient_direction, gradient_magnitude = self.fire_detector.extract_intensity_gradient(frame, mask)

        # Track detections using IOU
        iou_matches = []
        if not self.previous_detections.empty and not detections.empty:
            iou_matches = self.track_detections_with_iou(detections, self.previous_detections)

            # Store IOU scores in history
            avg_iou = np.mean([match[2] for match in iou_matches]) if iou_matches else 0
            self.iou_history.append(avg_iou)

            # Keep history limited
            if len(self.iou_history) > self.history_size:
                self.iou_history.pop(0)

            # Use IOU for better Kalman updates
            if iou_matches:
                self.update_kalman_with_best_match(iou_matches, detections, self.previous_detections)

        # Store current detections for next frame
        self.previous_detections = detections.copy()

        self.kalman.update(centroid)

        self.position_history.append(centroid)
        self.area_history.append(area)
        self.contour_history.append(contour if contour is not None else None)
        self.detections_history.append(detections)
        self.shape_history.append((shape_direction, shape_confidence))
        self.gradient_history.append((gradient_direction, gradient_magnitude))

        if len(self.position_history) > self.history_size:
            self.position_history.pop(0)
            self.area_history.pop(0)
            self.contour_history.pop(0)
            self.detections_history.pop(0)
            self.shape_history.pop(0)
            self.gradient_history.pop(0)

        if frame_index % 5 == 0 and len(self.position_history) >= 2:
            self._make_prediction(frame_index, centroid, area)

        self._evaluate_predictions(frame_index, centroid, area)

        return mask, detections

    def _make_prediction(self, frame_index, current_position, current_area):
        kalman_prediction = self.kalman.predict_future(self.prediction_window)[-1] if self.kalman.initialized else current_position

        linear_direction, growth_factor = self.predict_direction()

        linear_prediction = current_position
        if len(self.position_history) >= 2:
            oldest_pos = self.position_history[0]
            newest_pos = self.position_history[-1]
            dx = newest_pos[0] - oldest_pos[0]
            dy = newest_pos[1] - oldest_pos[1]
            steps_taken = len(self.position_history) - 1

            step_x = dx / steps_taken if steps_taken > 0 else 0
            step_y = dy / steps_taken if steps_taken > 0 else 0

            linear_x = current_position[0] + (step_x * self.prediction_window)
            linear_y = current_position[1] + (step_y * self.prediction_window)
            linear_prediction = (int(linear_x), int(linear_y))

        shape_prediction = current_position
        if self.shape_history and self.shape_history[-1][0] is not None:
            direction, confidence = self.shape_history[-1]
            shape_x = current_position[0] + direction[0] * confidence * self.prediction_window * 10
            shape_y = current_position[1] + direction[1] * confidence * self.prediction_window * 10
            shape_prediction = (int(shape_x), int(shape_y))

        gradient_prediction = current_position
        if self.gradient_history and self.gradient_history[-1][0] is not None:
            direction, magnitude = self.gradient_history[-1]
            gradient_x = current_position[0] + direction[0] * magnitude * self.prediction_window * 0.5
            gradient_y = current_position[1] + direction[1] * magnitude * self.prediction_window * 0.5
            gradient_prediction = (int(gradient_x), int(gradient_y))

        weights = self.feature_weights
        final_x = (weights['position'] * kalman_prediction[0] +
                  weights['shape'] * shape_prediction[0] +
                  weights['intensity'] * linear_prediction[0] +
                  weights['gradient'] * gradient_prediction[0])

        final_y = (weights['position'] * kalman_prediction[1] +
                  weights['shape'] * shape_prediction[1] +
                  weights['intensity'] * linear_prediction[1] +
                  weights['gradient'] * gradient_prediction[1])

        final_prediction = (int(final_x), int(final_y))

        prediction = {
            'frame_index': frame_index,
            'current_position': current_position,
            'current_area': current_area,
            'predicted_position': final_prediction,
            'kalman_prediction': kalman_prediction,
            'linear_prediction': linear_prediction,
            'shape_prediction': shape_prediction,
            'gradient_prediction': gradient_prediction,
            'evaluation_frame': frame_index + self.prediction_window,
            'evaluated': False
        }

        self.predictions.append(prediction)
        self.prediction_stats['total_predictions'] += 1

    def _calculate_dynamic_threshold(self, fire_area):
        if fire_area <= 0:
            return 50

        fire_width = np.sqrt(fire_area)
        threshold = max(30, min(200, fire_width * 0.1))
        return threshold

    def _evaluate_predictions(self, current_frame_index, current_position, current_area):
        for prediction in self.predictions:
            if not prediction['evaluated'] and current_frame_index >= prediction['evaluation_frame']:
                pred_x, pred_y = prediction['predicted_position']
                actual_x, actual_y = current_position

                distance_error = np.sqrt((pred_x - actual_x)**2 + (pred_y - actual_y)**2)

                fire_area = prediction['current_area']
                success_threshold = self._calculate_dynamic_threshold(fire_area)

                self.prediction_stats['thresholds_used'].append(success_threshold)

                success = distance_error <= success_threshold

                if distance_error <= success_threshold * 0.5:
                    quality = 'excellent'
                elif distance_error <= success_threshold:
                    quality = 'good'
                elif distance_error <= success_threshold * 1.5:
                    quality = 'moderate'
                else:
                    quality = 'poor'

                self.quality_categories[quality] += 1

                prediction['evaluated'] = True
                prediction['actual_position'] = current_position
                prediction['distance_error'] = distance_error
                prediction['success_threshold'] = success_threshold
                prediction['success'] = success
                prediction['quality'] = quality

                self.prediction_stats['prediction_errors'].append(distance_error)
                if success:
                    self.prediction_stats['successful_predictions'] += 1
                else:
                    self.prediction_stats['failed_predictions'] += 1

                total = self.prediction_stats['successful_predictions'] + self.prediction_stats['failed_predictions']
                if total > 0:
                    self.prediction_stats['success_rate'] = self.prediction_stats['successful_predictions'] / total

                self.prediction_results.append(prediction)

    def predict_direction(self):
        if len(self.position_history) < 2:
            return (0, 0), 1.0

        oldest_pos = self.position_history[0]
        newest_pos = self.position_history[-1]

        dx = newest_pos[0] - oldest_pos[0]
        dy = newest_pos[1] - oldest_pos[1]

        magnitude = np.sqrt(dx**2 + dy**2)

        if magnitude > 1e-6:
            dx, dy = dx/magnitude, dy/magnitude
        else:
            dx, dy = 0, 1

        if self.area_history[0] > 0:
            growth_factor = self.area_history[-1] / self.area_history[0]
        else:
            growth_factor = 1.2

        return (dx, dy), growth_factor

    def create_danger_zone(self, frame, prediction_distance=50, frame_index=None):
        if not self.contour_history or self.contour_history[-1] is None:
            return frame.copy(), np.zeros_like(frame[:,:,0])

        latest_contour = self.contour_history[-1]
        latest_center = self.position_history[-1]
        latest_detections = self.detections_history[-1]

        latest_area = self.area_history[-1] if self.area_history else 0
        dynamic_threshold = self._calculate_dynamic_threshold(latest_area)

        kalman_pos, kalman_vel, _ = self.kalman.get_state()

        vel_magnitude = np.sqrt(kalman_vel[0]**2 + kalman_vel[1]**2)
        if vel_magnitude > 1e-6:
            kalman_dx, kalman_dy = kalman_vel[0]/vel_magnitude, kalman_vel[1]/vel_magnitude
        else:
            kalman_dx, kalman_dy = 0, 0

        visualization = frame.copy()

        for _, detection in latest_detections.iterrows():
            x1, y1, x2, y2 = int(detection['xmin']), int(detection['ymin']), int(detection['xmax']), int(detection['ymax'])
            conf = detection['confidence']

            cv2.rectangle(visualization, (x1, y1), (x2, y2), (0, 0, 255), 2)

        if latest_contour is not None:
            cv2.drawContours(visualization, [latest_contour], -1, (0, 0, 255), 2)

        danger_mask = np.zeros_like(frame[:,:,0])

        kalman_predictions = self.kalman.predict_future(int(prediction_distance/5))

        prev_point = latest_center
        for i, point in enumerate(kalman_predictions):
            point = (int(point[0]), int(point[1]))
            cv2.line(visualization, prev_point, point, (255, 0, 0), 1 + i//3)
            prev_point = point

        linear_direction, growth_factor = self.predict_direction()
        shape_direction, shape_confidence = self.shape_history[-1] if self.shape_history else ((0,0), 0)
        gradient_direction, gradient_magnitude = self.gradient_history[-1] if self.gradient_history else ((0,0), 0)

        weights = self.feature_weights

        shape_dir_x = shape_direction[0] if shape_direction else 0
        shape_dir_y = shape_direction[1] if shape_direction else 0
        gradient_dir_x = gradient_direction[0] if gradient_direction else 0
        gradient_dir_y = gradient_direction[1] if gradient_direction else 0

        combined_dx = (weights['position'] * kalman_dx +
                      weights['shape'] * shape_dir_x +
                      weights['gradient'] * gradient_dir_x +
                      weights['intensity'] * linear_direction[0])

        combined_dy = (weights['position'] * kalman_dy +
                      weights['shape'] * shape_dir_y +
                      weights['gradient'] * gradient_dir_y +
                      weights['intensity'] * linear_direction[1])

        magnitude = np.sqrt(combined_dx**2 + combined_dy**2)
        if magnitude > 1e-6:
            combined_dx, combined_dy = combined_dx/magnitude, combined_dy/magnitude
        else:
            combined_dx, combined_dy = 0, 1

        danger_endpoint = (
            int(latest_center[0] + combined_dx * prediction_distance),
            int(latest_center[1] + combined_dy * prediction_distance)
        )

        angle = np.arctan2(combined_dy, combined_dx)
        base_width = np.sqrt(max(1, self.area_history[-1]) / np.pi) * growth_factor * 0.8

        p1 = latest_center
        p2 = (
            int(latest_center[0] + base_width * np.cos(angle + np.pi/2)),
            int(latest_center[1] + base_width * np.sin(angle + np.pi/2))
        )
        p3 = danger_endpoint
        p4 = (
            int(latest_center[0] + base_width * np.cos(angle - np.pi/2)),
            int(latest_center[1] + base_width * np.sin(angle - np.pi/2))
        )

        danger_zone_points = np.array([p1, p2, p3, p4], np.int32)
        danger_zone_points = danger_zone_points.reshape((-1, 1, 2))
        cv2.fillPoly(danger_mask, [danger_zone_points], 255)

        danger_overlay = visualization.copy()
        danger_overlay[danger_mask > 0] = (0, 255, 0)
        visualization = cv2.addWeighted(visualization, 0.7, danger_overlay, 0.3, 0)

        cv2.arrowedLine(
            visualization,
            latest_center,
            danger_endpoint,
            (0, 255, 0),
            3
        )

        if frame_index is not None:
            for prediction in self.predictions:
                if prediction['evaluation_frame'] == frame_index:
                    cv2.circle(
                        visualization,
                        prediction['predicted_position'],
                        10,
                        (0, 255, 255),
                        -1
                    )

                    try:
                        kalman_x = int(prediction['kalman_prediction'][0])
                        kalman_y = int(prediction['kalman_prediction'][1])
                        cv2.circle(
                            visualization,
                            (kalman_x, kalman_y),
                            8,
                            (255, 0, 0),
                            -1
                        )
                    except (TypeError, ValueError):
                        pass

                    cv2.putText(
                        visualization,
                        f"Predicted Position",
                        (prediction['predicted_position'][0] + 10, prediction['predicted_position'][1]),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 255, 255),
                        2
                    )

                    if 'success_threshold' in prediction:
                        cv2.circle(
                            visualization,
                            prediction['predicted_position'],
                            int(prediction['success_threshold']),
                            (255, 255, 0),
                            1
                        )

            if self.prediction_stats['total_predictions'] > 0:
                success_text = f"Prediction Accuracy: {self.prediction_stats['success_rate']*100:.1f}% ({self.prediction_stats['successful_predictions']}/{self.prediction_stats['total_predictions']})"
                cv2.putText(
                    visualization,
                    success_text,
                    (20, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

                if self.prediction_stats['thresholds_used']:
                    threshold_text = f"Dynamic Threshold: {self.prediction_stats['thresholds_used'][-1]:.1f} px"
                    cv2.putText(
                        visualization,
                        threshold_text,
                        (20, 60),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (255, 255, 255),
                        2
                    )

            # Display average IOU if available
            if self.iou_history:
                iou_text = f"Avg IOU: {self.iou_history[-1]:.2f}"
                cv2.putText(
                    visualization,
                    iou_text,
                    (20, 90),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

            cv2.putText(
                visualization,
                "Ceiling Lamp Filter: ACTIVE",
                (visualization.shape[1] - 280, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (255, 255, 255),
                2
            )

            # Add IOU tracking status
            cv2.putText(
                visualization,
                "IOU Tracking: ACTIVE",
                (visualization.shape[1] - 280, 60),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (255, 255, 255),
                2
            )

        return visualization, danger_mask

    def get_performance_metrics(self):
        metrics = self.fire_detector.get_performance_metrics()

        metrics.update({
            'total_predictions': self.prediction_stats['total_predictions'],
            'successful_predictions': self.prediction_stats['successful_predictions'],
            'failed_predictions': self.prediction_stats['failed_predictions'],
            'prediction_success_rate': self.prediction_stats['success_rate'],
        })

        if self.prediction_stats['prediction_errors']:
            metrics['avg_prediction_error'] = np.mean(self.prediction_stats['prediction_errors'])
            metrics['max_prediction_error'] = np.max(self.prediction_stats['prediction_errors'])
            metrics['min_prediction_error'] = np.min(self.prediction_stats['prediction_errors'])

        if self.prediction_stats['thresholds_used']:
            metrics['avg_threshold'] = np.mean(self.prediction_stats['thresholds_used'])
            metrics['min_threshold'] = np.min(self.prediction_stats['thresholds_used'])
            metrics['max_threshold'] = np.max(self.prediction_stats['thresholds_used'])

        if self.iou_history:
            metrics['avg_temporal_iou'] = np.mean(self.iou_history)
            metrics['max_temporal_iou'] = np.max(self.iou_history) if self.iou_history else 0
            metrics['min_temporal_iou'] = np.min(self.iou_history) if self.iou_history else 0

        metrics.update(self.quality_categories)

        return metrics


def process_video(video_path, output_path='tata.mp4', model_path='bestt.pt', window_size=5, prediction_window=10):
    output_dir = "fire_predictions_enhanced"
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video file: {video_path}")
        print(f"File exists: {os.path.exists(video_path)}")
        if os.path.exists(video_path):
            print(f"File size: {os.path.getsize(video_path)} bytes")
            print(f"File permissions: {oct(os.stat(video_path).st_mode)}")
        raise IOError(f"Error: Could not open video {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    predictor = FireSpreadPredictor(
        yolo_model_path=model_path,
        history_size=window_size,
        prediction_window=prediction_window
    )

    start_time = time.time()
    print(f"Processing video with enhanced prediction, ceiling lamp filtering, and IOU tracking... ")

    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    with tqdm(total=total_frames, desc="Progress", unit="%", bar_format='{l_bar}{bar}| {percentage:3.0f}%', ncols=80) as pbar:
        frame_idx = 0

        for _ in range(min(window_size, total_frames)):
            ret, frame = cap.read()
            if not ret:
                break
            timestamp = frame_idx / fps if fps > 0 else frame_idx
            predictor.add_frame(frame, frame_idx, timestamp)
            frame_idx += 1
            pbar.update(1)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            timestamp = frame_idx / fps if fps > 0 else frame_idx
            mask, _ = predictor.add_frame(frame, frame_idx, timestamp)

            prediction_distance = width * 0.15
            visualization, _ = predictor.create_danger_zone(frame, prediction_distance, frame_idx)

            out.write(visualization)

            frame_idx += 1
            if frame_idx % 5 == 0:
                pbar.update(5)

            if frame_idx % 30 == 0:
                gc.collect()
                torch.cuda.empty_cache() if torch.cuda.is_available() else None

    cap.release()
    out.release()

    total_time = time.time() - start_time

    metrics = predictor.get_performance_metrics()
    metrics['processing_time'] = total_time
    metrics['processing_fps'] = total_frames / total_time

    # Create report
    print(f"Processed {total_frames} frames in {total_time:.1f} seconds")
    print(f"Output video saved to {output_path}")

    # Print IOU metrics
    if 'avg_iou' in metrics:
        print(f"Average IOU: {metrics['avg_iou']:.3f}")
    if 'avg_temporal_iou' in metrics:
        print(f"Average Temporal IOU: {metrics['avg_temporal_iou']:.3f}")

    return metrics

def create_performance_report(metrics):
    """Create a performance report with metrics including IOU metrics."""
    print("\n--- PERFORMANCE REPORT ---\n")

    print("Detection Metrics:")
    print(f"  Total frames processed: {metrics.get('total_frames', 0)}")
    print(f"  Average inference time: {metrics.get('avg_inference_time', 0):.4f} seconds")
    print(f"  Processing FPS: {metrics.get('processing_fps', 0):.2f}")
    print(f"  Detection rate: {metrics.get('detection_rate', 0)*100:.2f}%")

    print("\nIOU Metrics:")
    print(f"  Average IOU: {metrics.get('avg_iou', 0):.4f}")
    print(f"  Min IOU: {metrics.get('min_iou', 0):.4f}")
    print(f"  Max IOU: {metrics.get('max_iou', 0):.4f}")
    print(f"  Average Temporal IOU: {metrics.get('avg_temporal_iou', 0):.4f}")


    print("\n--- END OF REPORT ---\n")

# Example usage
if __name__ == "__main__":
    try:
        # Mount Google Drive if in Colab
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            in_colab = True
        except:
            in_colab = False

        # Set paths
        if in_colab:
            model_path = '/content/drive/MyDrive/bestt.pt'
            video_path = '/content/drive/MyDrive/fire.mp4'
            output_path = '/content/drive/MyDrive/fire_prediction_with_iou.mp4'
        else:
            model_path = 'bestt.pt'
            video_path = 'fire.mp4'
            output_path = 'fire_prediction_with_iou.mp4'

        # Process video and generate metrics with enhanced prediction and IOU tracking
        metrics = process_video(
            video_path=video_path,
            output_path=output_path,
            model_path=model_path,
            window_size=5,
            prediction_window=10
        )

        # Create report with new IOU metrics
        create_performance_report(metrics)

    except Exception as e:
        print(f"Error: {e}")

Mounted at /content/drive
Initializing YOLOv8 detector with model: /content/drive/MyDrive/bestt.pt
Installing ultralytics...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.1/994.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:

Progress: 100%|██████████████████████████████████████████████████████████▉| 100%

Processed 3613 frames in 1822.3 seconds
Output video saved to /content/drive/MyDrive/fire_prediction_with_iou.mp4
Average IOU: 0.850
Average Temporal IOU: 0.821

--- PERFORMANCE REPORT ---

Detection Metrics:
  Total frames processed: 3613
  Average inference time: 0.3929 seconds
  Processing FPS: 1.98
  Detection rate: 98.12%
  Average confidence: 0.4068

IOU Metrics:
  Average IOU: 0.8500
  Min IOU: 0.2755
  Max IOU: 0.9997
  Average Temporal IOU: 0.8215

Prediction Metrics:
  Prediction success rate: 31.67%
  Total predictions: 722
  Successful predictions: 228
  Failed predictions: 492

Prediction Quality:
  Excellent: 66
  Good: 162
  Moderate: 211
  Poor: 281

--- END OF REPORT ---

